In [1]:
import os
import PIL
import numpy as np
import cv2
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms

import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

from jetbot import Robot, Camera, bgr8_to_jpeg
from mobile import MobileController

In [4]:

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

In [5]:
def _preprocess(image):
        image = PIL.Image.fromarray(image)
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image).to(device).half()
        image.sub_(mean[:, None, None]).div_(std[:, None, None])
        return image[None, ...]

In [6]:
# Initialize DL model

MODEL_PATH = 'best_steering_model_ResNet.pth'
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model.load_state_dict(torch.load(MODEL_PATH))
model = model.to(device)
model = model.eval().half()


In [7]:
#Initialize Jetbot
WHEEL_TRACK = 10
robot = Robot()
mobile_controller = MobileController(WHEEL_TRACK, robot)

In [8]:
# Prepare gage widgets.

speed = widgets.FloatSlider(min=-1.0, max=1.0, description='speed')
steering = widgets.FloatSlider(min=-1.0, max=1.0, description='steering')

traitlets.dlink((mobile_controller, 'speed'), (speed,'value'))
traitlets.dlink((mobile_controller, 'radius'), (steering, 'value'))

In [9]:
#Camera initialize
camera = Camera.instance(fps=5, width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera,'value'), (image,'value'), transform=bgr8_to_jpeg)

In [10]:
layout = widgets.Layout(width='100px', height='64px')

panel = widgets.VBox([speed, steering])
display(widgets.HBox([panel,image]))

In [11]:
def update(change):
    image = _preprocess(change['new'])
    xy = model(image).detach().float().cpu().numpy().flatten()
    slottle = xy[0]
    handle = xy[1]
    mobile_controller.set_control(float(slottle), float(handle))

update({'new':camera.value})


right :0.273828125, left :0.3911830357142857
right :0.4797149385724749, left :0.39777278900146484
right :0.4797149385724749, left :0.39777278900146484


In [12]:
camera.observe(update, names='value')

right :0.5608079774039132, left :0.4650139808654785
right :0.7101092849458966, left :0.6740358114242554
right :0.7101092849458966, left :0.6740358114242554
right :0.6641839742660522, left :0.6304435014724732
right :0.7477384898811579, left :0.747419823333621
right :0.7477384898811579, left :0.747419823333621
right :0.6862805318087339, left :0.6859880570322275
right :0.6808414444327354, left :0.6826043799519539
right :0.6808414444327354, left :0.6826043799519539
right :0.7108429864048957, left :0.7126836063606399
right :0.6613348126411438, left :0.677320625100817
right :0.6613348126411438, left :0.677320625100817
right :0.7045534253120422, left :0.7215839198657444
right :0.721455991268158, left :0.73365718126297
right :0.721455991268158, left :0.73365718126297
right :0.7717043638229371, left :0.7847553491592407
right :0.7933013498783111, left :0.8001817677702222
right :0.7933013498783111, left :0.8001817677702222
right :0.8861849009990692, left :0.8938709114279065
right :0.8756590315273

In [15]:
camera.unobserve(update, names='value')
robot.stop()